In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from PIL import Image, ImageDraw
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random

In [23]:
def drawBoundingRectangle(coordsTup , ImgObj, color='#ff0000', width=5, alpha=1.0):
    """
    Creates a box to highligt a location on the image object

    coordsTup = tuple of bbox corners. 
        top left coords = first 2 elements
        bottom right coords = last 2 elements
    width: width of outline
    color: hex value of color. Default red
    alpha: opacity. Range: 0,1
    """
    #Checking for transparency
    if alpha > 1 or alpha < 0: 
        alpha =1
    
    color_with_opacity = color + hex(int(alpha*255))[-2:]
    
    # Draw a rectangle
    draw = ImageDraw.Draw(ImgObj,'RGBA')
    
    p1_coord = coordsTup[0:2]
    p2_coord = coordsTup[2:4]
    draw.rectangle([p1_coord, p2_coord], outline=color_with_opacity, width=width)
    
    
def createNewPotentialBboxes(imgObj):
    img_width,  img_height = imgObj.size
    overlap = 0.4

    df_list = []
    

    for winsize in [ (277*0.75) , (277), (277*1.5)]:
        winsize = round(winsize)        
        
        if (winsize > img_width) or (winsize > img_height):
            break            
        
        stride = round(winsize*(1-overlap))
        #if winsize odd, increase by 1 pixel. Just easier
        if (winsize % 2) != 0:
            winsize+=1
        xvals = np.arange((winsize/2), (img_width-winsize/2),stride)
        yvals = np.arange((winsize/2), (img_height-winsize/2),stride)
        list_centroids = [(x,y) for x in xvals for y in yvals]

        list_coordTuples = [get_bboxCoords(x_cnt,y_cnt,winsize) for x_cnt,y_cnt in list_centroids]

        #Appends new records for current winsize and bbox
        df_list.append(pd.DataFrame([list_coordTuples,[winsize]*len(list_coordTuples)]).T)

    df_bboxPotentials = pd.concat(df_list,ignore_index=True,axis=0)
    df_bboxPotentials.columns = ['bbox_bounds','winsize']
    df_bboxPotentials.to_csv(f'mass_maps/bboxPotentials_{img_width}_{img_height}.csv',index = False)
    return df_bboxPotentials

def get_bboxCoords(x_cnt,y_cnt,winsize):
    return int(x_cnt - winsize/2),int(y_cnt - winsize/2),int(x_cnt + winsize/2),int(y_cnt + winsize/2)

def getPotentialBboxes(imgObj,forceCreateNew = False):
    
    img_width,  img_height = imgObj.size
    
    if forceCreateNew:
        print('force creating new')
        df_bboxPotentials = createNewPotentialBboxes(imgObj)
    else:
        try:
            print('reading current')
            df_bboxPotentials = pd.read_csv(f'mass_maps/bboxPotentials_{img_width}_{img_height}.csv')
        except FileNotFoundError:
            print('no existing doc. creating')
            df_bboxPotentials = createNewPotentialBboxes(imgObj)
        
        
    return df_bboxPotentials

def getClassFromPred(pred,threshold):
    if max(pred)>threshold :
        return pred.argmax()
    return -1


def getRandomColor():
    #Shamelessly stolen from one of the references above
    #Returns a random hex color value
    return str(["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])][0])


colorMap = {n:getRandomColor() for n in range(0,8)}

In [4]:
im1 = Image.open('mass_maps/ESP_073483_1580_RED.NOMAP.browse.jpg')
img_width,  img_height = im1.size

In [5]:
df_bbox = getPotentialBboxes(im1,forceCreateNew=True)

force creating new


In [6]:
df_bbox

,bbox_bounds,winsize
0,"(0, 0, 138, 138)",138
1,"(0, 166, 138, 304)",138
2,"(0, 332, 138, 470)",138
3,"(0, 498, 138, 636)",138
4,"(0, 664, 138, 802)",138
...,...,...
1582,"(1662, 6814, 2008, 7160)",346
1583,"(1662, 6980, 2008, 7326)",346
1584,"(1662, 7146, 2008, 7492)",346
1585,"(1662, 7312, 2008, 7658)",346


In [7]:
list_imagesToClassify = [im1.crop(df_bbox.iloc[rowNum]['bbox_bounds']) 
                         for rowNum in range(0,df_bbox.shape[0])]

In [31]:
final_model = tf.keras.models.load_model('final_finalModel__jk__butReally_FinalModel.h5py')

In [32]:
# Fresh start on sliced output folder
slicePath = f"{'/'.join(im1.filename.split('/')[:-1])}/_slices_{im1.filename.split('/')[-1]}/"
slicePath = slicePath.replace('.','_')

if (os.path.exists(slicePath) and os.path.isdir(slicePath)):
    shutil.rmtree(slicePath)
os.makedirs(slicePath)

for count, img in enumerate(list_imagesToClassify):
    img.save(''.join([slicePath,str(count),'_',(im1.filename.split('/')[-1])]))

In [33]:
# Normalizing data for 8 bit
predict_datagen = ImageDataGenerator(rescale=1./255)

print(slicePath)
predict_generator = predict_datagen.flow_from_directory(
    'mass_maps/',
    target_size = (227,227),
    batch_size=8,
    color_mode='rgb',
    class_mode=None
)
predict_generator.reset()

preds = final_model.predict(predict_generator)

mass_maps/_slices_ESP_073483_1580_RED_NOMAP_browse_jpg/
Found 1587 images belonging to 1 classes.
199/199 [==============================] - 7s 35ms/step


In [34]:
df_bbox['predClass'] = [getClassFromPred(pred,0.9) for pred in preds]

In [35]:
df_bbox[(df_bbox.predClass>-1) & (df_bbox.predClass!=4)]

,bbox_bounds,winsize,predClass
35,"(0, 5817, 138, 5955)",138,1
140,"(332, 7645, 470, 7783)",138,1
389,"(1329, 2160, 1467, 2298)",138,1
453,"(1495, 4986, 1633, 5124)",138,1
562,"(1828, 7478, 1966, 7616)",138,1
577,"(0, 2160, 278, 2438)",278,1
682,"(332, 3988, 610, 4266)",278,1
776,"(664, 3988, 942, 4266)",278,1
871,"(997, 4155, 1275, 4433)",278,1
957,"(1329, 2825, 1607, 3103)",278,1


In [ ]:
mask_img = Image.new("RGBA", (img_width, img_height), (0, 0, 0, 0))
df_validBbox = df_bbox[(df_bbox.predClass>-1) & (df_bbox.predClass!=4)][['bbox_bounds','predClass']]

for idx in range(0, df_validBbox.shape[0]):
    
    drawBoundingRectangle(df_validBbox.iloc[idx]['bbox_bounds'],
                          mask_img,
                          color=colorMap[df_validBbox.iloc[idx]['predClass']]
                                         )

im2 = im1.convert('RGB')

im2.paste(mask_img,(0,0),mask_img)
im2

In [ ]:
df_bbox.iloc[157]

In [ ]:
for each in list_imagesToClassify[157:158]:
    display(each)

In [ ]:
preds[157].argmax()

In [ ]:
max(preds[157])

In [ ]:
[getClassFromPred(pred,0.7) for pred in preds]

In [ ]:
for each in preds:
    print((each))
    print(len(each))
    print(each.argmax())
    print(max(each))
    break

In [ ]:
preds[0]

In [20]:
df_bbox[df_bbox.predClass.isin([1,2,3])]

,bbox_bounds,winsize,predClass
73,"(166, 4321, 304, 4459)",138,1
266,"(831, 5152, 969, 5290)",138,1
299,"(997, 2825, 1135, 2963)",138,1
421,"(1329, 7478, 1467, 7616)",138,1
476,"(1662, 997, 1800, 1135)",138,1
662,"(332, 664, 610, 942)",278,1
1045,"(1662, 1828, 1940, 2106)",278,1
1285,"(664, 3324, 1010, 3670)",346,1
1292,"(664, 4487, 1010, 4833)",346,1
1309,"(664, 7312, 1010, 7658)",346,1


In [37]:
im1.filename

'mass_maps/ESP_073483_1580_RED.NOMAP.browse.jpg'